# Image Mask Generator code:

## This code is used to read jpg images, use the SAM library created by Meta AI, select the best mask produced by SAM, adapt it to a correct label format for our CNN model, build a contour around the mask and finally save it as png format.

### Created by: Jorge Rodriguez
### Crated date: July 27th 2024
### Last Modified July 31th 2024



In [58]:
import numpy as np
import matplotlib.pyplot as plt

## Setting the correct Diretory Structure for Input Images and Targe Masks:

In [59]:
import os
input_dir = "./images/"
target_dir = "./annotations/trimaps/"
input_img_paths = sorted(
    [os.path.join(input_dir, fname)
for fname in os.listdir(input_dir) if fname.endswith(".jpg")])

In [60]:
target_paths = sorted(
    [os.path.join(target_dir, fname)
for fname in os.listdir(target_dir)
if fname.endswith(".png") and not fname.startswith(".")])


# Loading the pre-train SAM Model.

In [61]:
# ! wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
CHECKPOINT_PATH='./sam_vit_h_4b8939.pth'

In [62]:
import torch
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
MODEL_TYPE = "vit_h"

In [63]:
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor


sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH).to(device=DEVICE)


mask_generator = SamAutomaticMaskGenerator(sam)

## This function will get the masks from the image using SAM pre-train model

In [64]:
# Import opencv package
# pip3 install opencv-python
import cv2

def get_mask(image):
    # Convert to RGB format
    
    # Give the path of your image
    IMAGE_PATH= image

    # Read the image from the path
    image= cv2.imread(IMAGE_PATH)

    # Convert to RGB format
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Generate segmentation mask
    output_mask = mask_generator.generate(image_rgb)
    sam_results = output_mask
    print(".",end="")

    return sam_results

In [65]:
input_img_paths

['./images/adidas_100.jpg',
 './images/adidas_101.jpg',
 './images/adidas_102.jpg',
 './images/adidas_103.jpg',
 './images/adidas_104.jpg',
 './images/adidas_105.jpg',
 './images/adidas_106.jpg',
 './images/adidas_107.jpg',
 './images/adidas_108.jpg',
 './images/adidas_109.jpg',
 './images/adidas_110.jpg',
 './images/adidas_111.jpg',
 './images/adidas_112.jpg',
 './images/adidas_113.jpg',
 './images/adidas_114.jpg',
 './images/adidas_115.jpg',
 './images/adidas_116.jpg',
 './images/adidas_117.jpg',
 './images/adidas_118.jpg',
 './images/adidas_119.jpg',
 './images/adidas_120.jpg',
 './images/adidas_121.jpg',
 './images/adidas_122.jpg',
 './images/adidas_123.jpg',
 './images/adidas_124.jpg',
 './images/adidas_125.jpg',
 './images/adidas_126.jpg',
 './images/adidas_127.jpg',
 './images/adidas_128.jpg',
 './images/adidas_129.jpg',
 './images/adidas_130.jpg',
 './images/adidas_131.jpg',
 './images/adidas_132.jpg',
 './images/adidas_133.jpg',
 './images/adidas_134.jpg',
 './images/adidas_13

## Adding the correct directory and extnesion file for the mask.

In [66]:
target_dir+input_img_paths[0].split("/")[-1].split(".")[0]+".png"

'./annotations/trimaps/adidas_100.png'

In [67]:
target_paths

[]

# Finding the Best Image Mask

In [68]:
def find_the_best_mask(sam_results):
    a = []
    for val in sam_results:
        #print((val['predicted_iou']))
        a.append(val['predicted_iou'])
    ar = np.array(a)
    mask = sam_results[np.argmax(ar)]['segmentation']
    return mask



## Saving The Mask

This Function gets the best mask, then builds the contour aorund the mask wiht the correct label, change the original mask from boolean to numeric, changes the background and finally saves it wiht the correct format.

In [69]:
def save_the_mask(mask, image_name):
    #target_dir
    # Change from Bool to uint8
    new_mask = mask.astype(np.uint8)
    
    # Put a Contour on the row side
    print("R",end='')
    for row in range(0, new_mask.shape[0]):
        #print("R",end='')
        for col in range(0, new_mask.shape[1]-1):
            #print(new_mask[row,col],end='')
            #print(previous_pixel)
            if col != new_mask.shape[1]-1:
                next_pixel = new_mask[row,col+1]
            #print(" n:",row,",",col+1," ",next_pixel)
            #print("[",new_mask[row,col],"-",next_pixel,"]",end='')
            if (new_mask[row,col] == 0) & (next_pixel == 1):
                new_mask[row,col] = 3
                #print("change")
            if (new_mask[row,col] == 1) & (next_pixel == 0):
                #print("change")
                new_mask[row,col+1] = 3

    # put a contour on the column side
    print("C",end='')
    for col in range(0, new_mask.shape[1]):
        #print("C",end='')
        for row in range(0, new_mask.shape[0]-1):
            #print(new_mask[row,col],end='')
            #print(previous_pixel)
            if row != new_mask.shape[0]-1:
                next_pixel = new_mask[row+1,col]
            #print(" n:",row,",",col+1," ",next_pixel)
            #print("[",new_mask[row,col],"-",next_pixel,"]",end='')
            if (new_mask[row,col] == 0) & ((next_pixel == 4) | (next_pixel == 1)):
                new_mask[row,col] = 3
                #print("change")
            if ((new_mask[row,col] == 4) | (new_mask[row,col] == 1)) & (next_pixel == 0):
                #print("change")
                new_mask[row+1,col] = 3
    
    # we need to change the backgrond from 0 to 2
    print("B",end='')
    for row in range(0, new_mask.shape[0]):
        for col in range(0, new_mask.shape[1]):
            if (new_mask[row,col] == 0):
                new_mask[row,col] = 2
        
    mask_image = (new_mask).astype(np.uint8)  # Convert to uint8 format

    # Save the mask image
    print("S",end='')
    png_mask = target_dir+image_name.split("/")[-1].split(".")[0]+".png"
    cv2.imwrite(png_mask, mask_image)
    return png_mask, mask_image

## This fucion is only for humans to see the mask image as the mask only has values of 1,2 and 3. if we do not normalize it, it is just a black image.

In [70]:
def display_target(target_array):
    normalized_array = (target_array.astype("uint8") - 1) * 127 
    plt.axis("off")
    plt.imshow(normalized_array[:, :])
    plt.show()

#img = img_to_array(load_img(target_paths[0], color_mode="grayscale"))
#display_target(new_mask)

## This funcion will take all the images one by one and return the mask image and save the files.

In [71]:
total_images = len(input_img_paths)
for i in range(len(input_img_paths)):
    print("processin image ",i," of ",total_images)
    print(f"Processing image {input_img_paths[i]}")
    sam_results = get_mask(input_img_paths[i])
    mask = find_the_best_mask(sam_results)
    png_mask, final_mask = save_the_mask(mask, input_img_paths[i])
    #display_target(final_mask)
    print(f"Saved mask {png_mask}")

processin image  0  of  711
Processing image ./images/adidas_100.jpg
.RCBSSaved mask ./annotations/trimaps/adidas_100.png
processin image  1  of  711
Processing image ./images/adidas_101.jpg
.RCBSSaved mask ./annotations/trimaps/adidas_101.png
processin image  2  of  711
Processing image ./images/adidas_102.jpg
.RCBSSaved mask ./annotations/trimaps/adidas_102.png
processin image  3  of  711
Processing image ./images/adidas_103.jpg
.RCBSSaved mask ./annotations/trimaps/adidas_103.png
processin image  4  of  711
Processing image ./images/adidas_104.jpg
.RCBSSaved mask ./annotations/trimaps/adidas_104.png
processin image  5  of  711
Processing image ./images/adidas_105.jpg
.RCBSSaved mask ./annotations/trimaps/adidas_105.png
processin image  6  of  711
Processing image ./images/adidas_106.jpg
.RCBSSaved mask ./annotations/trimaps/adidas_106.png
processin image  7  of  711
Processing image ./images/adidas_107.jpg
.RCBSSaved mask ./annotations/trimaps/adidas_107.png
processin image  8  of  